In [1]:
import os # define device ids to use before importing torch

import torch
import random
import numpy as np

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

In [23]:
import glob

#영문 Task, 한국어일 경우 lower 필요 x
def read_text_files(directory_path):
    text_data = []
    for filepath in glob.glob(os.path.join(directory_path, '*.txt')):
        with open(filepath, 'r', encoding='utf-8') as file:
            #모든 문자 소문자 변환
            text_data.append(file.read().lower())
    return text_data

pos_texts = read_text_files('./train/pos')
neg_texts = read_text_files('./train/neg')

pos_transformed = [[item, '1'] for item in pos_texts]
neg_transformed = [[item, '0'] for item in neg_texts]

In [24]:
test_pos_texts = read_text_files('./test/pos')
test_neg_texts = read_text_files('./test/neg')

test_pos_transformed = [[item, '1'] for item in test_pos_texts]
test_neg_transformed = [[item, '0'] for item in test_neg_texts]

In [29]:
import re

tokenizer = lambda x: x.split()

train_data = pos_transformed + neg_transformed
test_data = test_pos_transformed + test_neg_transformed

tokenized_train_dataset = []
tokenized_test_dataset = []
#print(train_texts[:2])


for data in train_data:
    #한국어일 경우
    #text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', ' ', text)
    text = re.sub(r'[,.!?;:()\"\'-]', ' ', data[0])
    tokens = tokenizer(text)
    labels = data[1]
    tokenized_train_dataset.append((tokens, labels))

for data in test_data:
    text = re.sub(r'[,.!?;:()\"\'-]', ' ', data[0])
    #한국어일 경우
    #text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', ' ', text)
    tokens = tokenizer(text)
    labels = data[1]
    tokenized_test_dataset.append((tokens, labels))


print(tokenized_train_dataset[:50])
print(tokenized_test_dataset[:50])

[(['this', 'is', 'one', 'of', 'my', 'favorite', 'movies', 'of', 'all', 'time', 'it', 's', 'great', 'and', 'the', 'acting', 'is', 'brilliant', 'in', 'the', 'scene', 'in', 'which', 'michael', 'caine', 'calls', 'the', 'police', 'in', 'tears', 'and', 'then', 'stops', 'the', 'waterworks', 'the', 'second', 'he', 'finishes', 'the', 'call', 'really', 'displays', 'caine', 's', 'brilliance', 'the', 'twists', 'are', 'a', 'lot', 'of', 'fun', 'the', 'film', 'is', 'top', 'notch'], '1'), (['i', 'm', 'giving', 'it', 'only', '9', 'out', 'of', '10', 'because', 'i', 'need', 'to', 'view', 'it', 'again', 'in', 'a', 'more', 'current', 'mind', 'frame', 'to', 'have', 'a', 'fresh', 'perspective', 'what', 'i', 'remember', 'is', 'an', 'amazing', 'psychological', 'thriller', 'that', 'dazzled', 'my', 'brain', 'and', 'my', 'eyes', 'when', 'i', 'watched', 'it', 'i', 'would', 'watch', 'it', 'every', 'time', 'it', 'came', 'on', 'hbo', 'back', 'in', 'the', 'day', 'probably', '1990', 'or', 'so', 'i', 'might', 'have', 'e

In [8]:
# make word2vec train data
from collections import Counter

token_counter = Counter()

for tokens, _ in tokenized_train_dataset:
    token_counter.update(tokens)

# remove tokens that appear only twice or less
min_count = 2
cleaned_vocab = {"[PAD]":0, "[UNK]":1}
cleaned_vocab_idx = 2

for token, count in token_counter.items():
    if count > min_count:
        cleaned_vocab[token] = cleaned_vocab_idx
        cleaned_vocab_idx += 1

In [9]:
# make word2vec train data
word2vec_train_datas = []
for train_text, _ in tokenized_train_dataset:
    word2vec_train_datas.append([word for word in train_text])

word2vec_train_datas[:5]

[['this',
  'is',
  'one',
  'of',
  'my',
  'favorite',
  'movies',
  'of',
  'all',
  'time',
  'it',
  's',
  'great',
  'and',
  'the',
  'acting',
  'is',
  'brilliant',
  'in',
  'the',
  'scene',
  'in',
  'which',
  'michael',
  'caine',
  'calls',
  'the',
  'police',
  'in',
  'tears',
  'and',
  'then',
  'stops',
  'the',
  'waterworks',
  'the',
  'second',
  'he',
  'finishes',
  'the',
  'call',
  'really',
  'displays',
  'caine',
  's',
  'brilliance',
  'the',
  'twists',
  'are',
  'a',
  'lot',
  'of',
  'fun',
  'the',
  'film',
  'is',
  'top',
  'notch'],
 ['i',
  'm',
  'giving',
  'it',
  'only',
  '9',
  'out',
  'of',
  '10',
  'because',
  'i',
  'need',
  'to',
  'view',
  'it',
  'again',
  'in',
  'a',
  'more',
  'current',
  'mind',
  'frame',
  'to',
  'have',
  'a',
  'fresh',
  'perspective',
  'what',
  'i',
  'remember',
  'is',
  'an',
  'amazing',
  'psychological',
  'thriller',
  'that',
  'dazzled',
  'my',
  'brain',
  'and',
  'my',
  'eyes'

In [10]:
from gensim.models import Word2Vec

# call CBOW or SkipGram
# CBOW_W2V = Word2Vec(sentences = word2vec_train_datas, vector_size = 32, window = 5, min_count = 1, workers = 4, sg = 0)
SkipGram_W2V = Word2Vec(sentences = word2vec_train_datas, vector_size = 32, window = 5, min_count = 1, workers = 4, sg = 1)

In [11]:
import numpy as np

# make embedding lookup matrix
embedding_list = []

for token, idx in cleaned_vocab.items():
    if token in SkipGram_W2V.wv:
        embedding_list.append(SkipGram_W2V.wv[token])
    elif token == "[PAD]":
        embedding_list.append(np.zeros(SkipGram_W2V.wv.vectors.shape[1]))
    elif token == "[UNK]":
        embedding_list.append(np.random.uniform(-1, 1, SkipGram_W2V.wv.vectors.shape[1]))
    else:
        embedding_list.append(np.random.uniform(-1, 1, SkipGram_W2V.wv.vectors.shape[1]))

embedding_loopup_matrix = np.vstack(embedding_list)

print(embedding_loopup_matrix.shape)    # (43354, 32)
print(len(cleaned_vocab))               # 43354

(39552, 32)
39552


In [12]:
from torch.utils.data import Dataset, DataLoader

# define dataset class
class SentimentDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label = int(self.data[index][1])
        tokens = self.data[index][0]

        token_ids = [self.vocab[token] if token in self.vocab else 1 for token in tokens]
        
        if len(token_ids) > 100:
            token_ids = token_ids[:100]
        else:
            token_ids = token_ids[:100] + [0] * (100 - len(token_ids))

        return torch.tensor(token_ids), torch.tensor(label)

In [13]:
import lightning as pl

class SentimentClassifierPL(pl.LightningModule):
    def __init__(self, sentiment_classifier):
        super(SentimentClassifierPL, self).__init__()
        self.model = sentiment_classifier
        self.loss = nn.CrossEntropyLoss()
        
        self.validation_step_outputs = []
        self.test_step_outputs = []
        self.save_hyperparameters()
    
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.loss(outputs, labels)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.loss(outputs, labels)
        self.log("val_loss", loss)
        self.validation_step_outputs.append((loss, outputs, labels))
        return loss, outputs, labels
    
    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        
        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("val_accuracy", accuracy)
        self.validation_step_outputs.clear()
    
    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.loss(outputs, labels)
        self.log("test_loss", loss)
        self.test_step_outputs.append((loss, outputs, labels))
        return loss, outputs, labels
    
    def on_test_epoch_end(self):
        outputs = self.test_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_test_loss", avg_loss)
        
        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("test_accuracy", accuracy)
        self.test_step_outputs.clear()
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

In [16]:
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelSummary

wandb.login()

def check_vocab_properties(vocab):
    print(f"Vocab size: {len(vocab)}")
    print(f"Vocab items: {list(vocab.items())[:5]}")


def check_performance(model, vocab,train_data, test_data, wandb_log_name):
    wandb.init(settings=wandb.Settings(start_method="fork"))
    wandb_logger = WandbLogger(project="article", name=wandb_log_name, group="round1_1")
    #wandb.init(project="NLP", entity="entityname")
    #wandb.config = params

    pl_model = SentimentClassifierPL(model)

    train_dataset = SentimentDataset(train_data, vocab)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_dataset = SentimentDataset(test_data, vocab)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    test_dataset = SentimentDataset(test_data, vocab)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

    trainer = pl.Trainer(
        max_epochs=3,
        accelerator="gpu",
        logger=wandb_logger,
        callbacks=[ModelSummary(max_depth=2)]
    )

    trainer.fit(
        model=pl_model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )

    trainer.test(dataloaders=test_loader)

    wandb.finish()

In [17]:
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_size):
        super(TextCNN, self).__init__()
        self.SG_embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_loopup_matrix), freeze=True)
        self.RD_embedding = nn.Embedding(vocab_size, 32)
        
        self.SG_conv1 = nn.Conv2d(1, 32, (3, 32))
        self.SG_conv2 = nn.Conv2d(1, 32, (4, 32))
        self.SG_conv3 = nn.Conv2d(1, 32, (5, 32))
        
        self.RD_conv1 = nn.Conv2d(1, 32, (3, 32))
        self.RD_conv2 = nn.Conv2d(1, 32, (4, 32))
        self.RD_conv3 = nn.Conv2d(1, 32, (5, 32))
        
        self.fc = nn.Linear(6*32, 2)

        
    def forward(self, x):
        SG_embedding = self.SG_embedding(x).unsqueeze(1)
        RD_embedding = self.RD_embedding(x).unsqueeze(1)

        SG_conv1_feature = F.relu(self.SG_conv1(SG_embedding).squeeze(3))
        SG_conv2_feature = F.relu(self.SG_conv2(SG_embedding).squeeze(3))
        SG_conv3_feature = F.relu(self.SG_conv3(SG_embedding).squeeze(3))
                                                                         
        RD_conv1_feature = F.relu(self.RD_conv1(RD_embedding).squeeze(3))
        RD_conv2_feature = F.relu(self.RD_conv2(RD_embedding).squeeze(3))
        RD_conv3_feature = F.relu(self.RD_conv3(RD_embedding).squeeze(3))

        SG_max1 = F.max_pool1d(SG_conv1_feature, SG_conv1_feature.size(2)).squeeze(2)  
        SG_max2 = F.max_pool1d(SG_conv2_feature, SG_conv2_feature.size(2)).squeeze(2)  
        SG_max3 = F.max_pool1d(SG_conv3_feature, SG_conv3_feature.size(2)).squeeze(2)  

        RD_max1 = F.max_pool1d(RD_conv1_feature, RD_conv1_feature.size(2)).squeeze(2)  
        RD_max2 = F.max_pool1d(RD_conv2_feature, RD_conv2_feature.size(2)).squeeze(2)  
        RD_max3 = F.max_pool1d(RD_conv3_feature, RD_conv3_feature.size(2)).squeeze(2)  

        x = torch.cat([SG_max1, SG_max2, SG_max3, RD_max1, RD_max2, RD_max3], dim=1)

        x = self.fc(x)

        return x

In [19]:
textcnn_model = TextCNN(len(cleaned_vocab))

In [20]:
sample_inputs = torch.randint(0, len(cleaned_vocab), (2, 100))

In [22]:
textcnn_model.forward(sample_inputs)

tensor([[0.3571, 0.2828],
        [0.1000, 0.2587]], grad_fn=<AddmmBackward0>)

In [27]:
check_performance(textcnn_model, cleaned_vocab, tokenized_train_dataset, tokenized_test_dataset, "task1")

/home/alki/.conda/envs/alki/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'sentiment_classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['sentiment_classifier'])`.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precis

Epoch 2: 100%|██████████| 391/391 [00:06<00:00, 59.40it/s, v_num=c5xq]     

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 391/391 [00:06<00:00, 59.09it/s, v_num=c5xq]


/home/alki/.conda/envs/alki/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at ./article/g7x5c5xq/checkpoints/epoch=2-step=1173.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./article/g7x5c5xq/checkpoints/epoch=2-step=1173.ckpt


Testing DataLoader 0: 100%|██████████| 391/391 [00:01<00:00, 368.79it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_loss         0.41524550318717957
      test_accuracy         0.8070799708366394
        test_loss            0.41533362865448
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


avg_test_loss,▁
avg_val_loss,█▃▁
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆█
test_accuracy,▁
test_loss,▁
train_loss,█▇▇▄▃▆▃▄▃▅▄▃▄▂▁▂▃▂▄▂▄▂▁
trainer/global_step,▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇████
val_accuracy,▁▆█
val_loss,█▃▁
avg_test_loss,0.41525
avg_val_loss,0.41525


In [28]:
wandb.finish()